In [4]:
#from selenium.webdriver import (Chrome, Firefox)
import time
import requests
from bs4 import BeautifulSoup
import pymongo

In [5]:
"""url = "https://www.leafly.com/hybrid/blue-dream/reviews"
url2 = "https://www.leafly.com/hybrid/blue-dream/reviews?page=2"
sel = "ul#strain-reviews-container"
#sel = 'button#btc-button btc-button--primary'
#<button class="btc-button btc-button--primary" tabindex="0" id="tou-continue">Yes</button>"""

'url = "https://www.leafly.com/hybrid/blue-dream/reviews"\nurl2 = "https://www.leafly.com/hybrid/blue-dream/reviews?page=2"\nsel = "ul#strain-reviews-container"\n#sel = \'button#btc-button btc-button--primary\'\n#<button class="btc-button btc-button--primary" tabindex="0" id="tou-continue">Yes</button>'

In [6]:
mc = pymongo.MongoClient()  # Connect to the MongoDB server using default settings
db = mc['strain_reviews']  # Use (or create) a database called 'election_predictions'
docs = db['review']  # Use (or create) a collection called 'docs'

In [7]:
docs.count_documents(filter={})

0

In [8]:
docs.drop()

In [9]:
def get_url(x,y, start=1):
    """X= number of urls to grab, y= strain name or other thing we are scraping 
    Pull in urls from a 'Load More' button and the like. Create identical URL's with differing numbers"""
    urls = []
    url = start
    while url <= x:
        urls.append("https://www.leafly.com/{}/reviews?page={}".format(y, url))
        url += 1 
    return urls

In [10]:
def get_reviews(l):
    """Pass in a list of URL's and return them in a mongo db table as a dicitonary with 
    {'url', 'html'} and their corresponding values"""
    htmls = []
    for url in l:
        r = requests.get(url)
        html = (r.content)
        htmls.append(html)
    table_check(l)
    results = dict(zip(l, htmls))
    return results

In [11]:
def table_check(l):
    """This checks to see if a url is already in my database of reviews. 
    Only needs to be run in the get reviews function"""
    for url in l:
        if docs.find_one({'url':url}):
            None
        else:
            docs.insert_one({'url': url,
                 'html': requests.get(url).content
                 })

In [12]:
def parse_docs(d):
    """Parse the HTML docs that we have stored in a dictionary, return as a list.
    Also scrape and parse star rating for each review """
    strain_text= []
    star_num = []
    for key, values in d.items():
        #stars = []
        soup = BeautifulSoup(values, 'html.parser')
        revs = soup.find_all('p',class_='strain-review__text') 
        for r in revs:
            strain_text.append(r.text)
        tags = soup.select("div.div.stars")
        for t in tags:
            stars.append(t.attrs['style'])
        star_num.append(star_int_conv(stars))
    return star_num, strain_text

In [13]:
def get_stars_list(d):
    stars = []
    for key, values in d.items():
        soup = BeautifulSoup(values, 'html.parser')
        tags = soup.select("div.div.stars")
        for t in tags:
            stars.append(t.attrs['style'])
    return stars

In [28]:
def star_int_conv(l):
    flat_stars= []
    star_num = []
    for s in l:
        star = int((s[6:].split(';')[0]).strip('px'))
        star_num.append((star/22))
    return star_num

In [15]:
# Works on the main explore page, but doesnt when i use filter...
#need to figure out how to get the filters to work and still scrapy.

def list_o_strains(i):
    LOS = []
    r = requests.get(i)
    soup2 = BeautifulSoup(r.content, 'html.parser')
    strains = soup2.find_all('a', class_="ga_Explore_Strain_Tile")
    for s in strains:
        LOS.append(s.attrs['href'])
    return LOS

# OVERVIEW of how CODE should Work

In [15]:
"""#Here is how i want the scrape to work
LOS = [List of Strains]
    #Pass in a list of the Strain names to scrape, pass strains through a for loop function that generates100 or so review pages,
    for s in LOS:
    url = "https://www.leafly.com/{}/reviews?page=".format(s)
    for i in range(1,100):
        d = get_reviews(url+str(i))
        st, rev = parse_docs(d)
        if len(st) > 0:
           append.(add to your dictionary)
        else:
            break"""

SyntaxError: invalid syntax (<ipython-input-15-0626370e37bb>, line 2)

## blue_dream_reviews

In [19]:
blue_dream_reviews = get_url(5, 'hybrid/blue-dream')

In [20]:
blue_dream_reviews

['https://www.leafly.com/hybrid/blue-dream/reviews?page=1',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=2',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=3',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=4',
 'https://www.leafly.com/hybrid/blue-dream/reviews?page=5']

In [21]:
bdr = get_reviews(blue_dream_reviews)

In [22]:
len(bdr)

5

In [23]:
stars = get_stars_list(bdr)

In [29]:
star_int_conv(stars)

[5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 4.0,
 4.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0]

In [251]:
#len(star_int_conv(stars))

25948800

In [16]:
st, reviews = parse_docs(bdr)

NameError: name 'stars' is not defined

In [ ]:
len(reviews)

In [274]:
st[0]

[5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0,
 5.0,
 5.0,
 1.0,
 4.0,
 5.0,
 4.0,
 1.0,
 3.0

In [255]:
len(st)

55

In [234]:
flat_stars = []
for sublist in st:
    for item in sublist:
        flat_stars.append(item)
print(len(flat_stars)

15264


## List of Strains

In [20]:
test_strains = list_o_strains('https://www.leafly.com/explore')

In [21]:
test_strains

['/hybrid/blue-dream',
 '/sativa/sour-diesel',
 '/hybrid/gsc',
 '/sativa/green-crack',
 '/hybrid/og-kush',
 '/indica/granddaddy-purple',
 '/hybrid/original-glue',
 '/hybrid/white-widow',
 '/hybrid/blue-dream',
 '/sativa/sour-diesel',
 '/hybrid/gsc',
 '/sativa/green-crack',
 '/hybrid/og-kush',
 '/indica/granddaddy-purple',
 '/hybrid/original-glue',
 '/hybrid/white-widow',
 '/sativa/jack-herer',
 '/indica/bubba-kush',
 '/hybrid/pineapple-express',
 '/hybrid/trainwreck',
 '/hybrid/ak-47',
 '/sativa/durban-poison',
 '/indica/northern-lights',
 '/hybrid/headband',
 '/indica/blue-cheese',
 '/sativa/strawberry-cough',
 '/hybrid/chemdawg',
 '/indica/purple-kush',
 '/sativa/lemon-haze',
 '/sativa/super-lemon-haze',
 '/indica/grape-ape',
 '/indica/blueberry',
 '/sativa/alaskan-thunder-fuck',
 '/sativa/super-silver-haze',
 '/indica/blackberry-kush',
 '/hybrid/cherry-pie',
 '/indica/master-kush',
 '/hybrid/skywalker-og',
 '/hybrid/cheese',
 '/indica/death-star',
 '/sativa/chocolope',
 '/sativa/amn

In [18]:
hybrid_url = "https://www.leafly.com/explore#/explore/category-hybrid"

In [19]:
def list_o_strains_hybrid(i):
    LOS = []
    r = requests.get(i)
    hy_soup = BeautifulSoup(r.content, 'html.parser')
    strains = hy_soup.find_all('a', class_="ng_scope")
    for s in strains:
        LOS.append(s.attrs['href'])
    return LOS

In [20]:
list_o_strains_hybrid(hybrid_url)

[]

# Full Function

In [1]:
def get_reviews2(l):
    """Pass in a list of URL's and return them in a mongo db table as a dicitonary with 
    {'url', 'html'} and their corresponding values"""
    r = requests.get(l)
    html = (r.content)
    return html

In [23]:
def parse_docs2(d):
    """Parse the HTML docs that we have stored in a dictionary, return as a list.
    Also scrape and parse star rating for each review """
    strain_text= []
    star_num = []
    soup = BeautifulSoup(d, 'html.parser')
    revs = soup.find_all('p',class_='strain-review__text') 
    for r in revs:
        strain_text.append(r.text)
    tags = soup.select("div.div.stars")
    stars = []
    for t in tags:
        stars.append(t.attrs['style'])
    star_num.append(star_int_conv(stars))
    return star_num, strain_text

In [30]:
def table_check2(l):
    """This checks to see if a url is already in my database of reviews. 
    Only needs to be run in the get reviews function"""
    for url in l:
        if docs.find_one({'url':url}):
            None
        else:
            docs.insert_one({'url': url,
                 'html': requests.get(url).content
                 })

In [24]:
def scraper_dummy(los):
    los_urls = []
    rev_urls = []
    for s in los:
        url = "https://www.leafly.com{}/reviews?page=".format(s)
        print(url)
        for i in range(1,100):
            rev_urls=url+str(i)
            d = get_reviews2(rev_urls)
            star, reviews = parse_docs2(d)
            print(len(star))
            if len(star) == 0:
                break
            #do something with (star, reviews)
            

In [27]:
scraper_dummy(test_strains)

https://www.leafly.com/hybrid/blue-dream/reviews?page=


NameError: name 'sublist' is not defined

In [44]:
docs.count_documents(filter={})

396

In [45]:
docs.find({})